# Invoking SageMarker-Spark Trained XGBoost Multi-Class Classification Model With Boto3 
_**Invoking your sagemaker-spark SDK trained model from boto3 to show how it can be leveraged in a web or mobile app**_

---

## Introduction


This notebook demonstrates how you can invoke your SageMaker-Spark trained XGBoost model deployed in the `MNIST-xgboost-train.ipynb` notebook.

---

## Download Test Record

For the purposes of this example we are downloading an existing test record that has already been converted to libsvm format.

In [3]:
import urllib.request

url = 'https://raw.githubusercontent.com/robperc/sagemaker-spark/master/examples/notebooks/jupyter/xgboost/test.data'
response = urllib.request.urlopen(url)
data = response.read().decode('utf-8') # This record's true label is 7.0
print(data)

## Prep Data

As you can see this is a very sparse dataset, meaning most of the features are `0.0`. The model endpoint expects the sparse entries to be removed before invocation. A simple approach to this data preparation problem is to treat the features as strings and remove any feature that ends with '0.0'.

In [5]:
# Naive way of removing sparse elements of

sparse_data = ' '.join([i for i in data.split(' ') if not i.endswith('0.0')])
sparse_data

'203:0.328125 204:0.72265625 205:0.62109375 206:0.58984375 207:0.234375 208:0.140625 231:0.8671875 232:0.9921875 233:0.9921875 234:0.9921875 235:0.9921875 236:0.94140625 237:0.7734375 238:0.7734375 239:0.7734375 240:0.7734375 241:0.7734375 242:0.7734375 243:0.7734375 244:0.7734375 245:0.6640625 246:0.203125 259:0.26171875 260:0.4453125 261:0.28125 262:0.4453125 263:0.63671875 264:0.88671875 265:0.9921875 266:0.87890625 267:0.9921875 268:0.9921875 269:0.9921875 270:0.9765625 271:0.89453125 272:0.9921875 273:0.9921875 274:0.546875 292:0.06640625 293:0.2578125 294:0.0546875 295:0.26171875 296:0.26171875 297:0.26171875 298:0.23046875 299:0.08203125 300:0.921875 301:0.9921875 302:0.4140625 327:0.32421875 328:0.98828125 329:0.81640625 330:0.0703125 354:0.0859375 355:0.91015625 356:0.99609375 357:0.32421875 382:0.50390625 383:0.9921875 384:0.9296875 385:0.171875 409:0.23046875 410:0.97265625 411:0.9921875 412:0.2421875 437:0.51953125 438:0.9921875 439:0.73046875 440:0.01953125 464:0.03515625 

## Invoke Endpoint

Now that our data has been prepared we can invoke the endpoint. Be sure to replace `endpoint_name` with the name of the endpoint created in the previous `MNIST-xgboost-train.ipynb` notebook (or feel free to use an alternative XGBoost MNIST classification endpoint you've created).

In [10]:
import boto3

runtime_client = boto3.client('runtime.sagemaker')

endpoint_name = 'name_of_endpoint_created_in_training_notebook'

payload = sparse_data.encode('utf-8').strip()

response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/x-libsvm', 
                                   Body=payload)
result = response['Body'].read().decode('ascii')
print('Predicted label is {}.'.format(result))

Predicted label is 7.0.
